In [3]:
pip install torch transformers peft

  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached markupsafe-3.0.3-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
Using cached torch-2.9.1-cp313-cp313-


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:


import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from peft import PeftModel
import os

# --- CONFIGURATION ---
NER_TAGS = ['O', 'B-Location', 'I-Location', 'B-Person', 'I-Person', 'B-Organization', 'I-Organization']
id2label = {i: tag for i, tag in enumerate(NER_TAGS)}
label2id = {tag: i for i, tag in enumerate(NER_TAGS)}

MODEL_CHECKPOINT = "xlm-roberta-base"
LOCAL_MODEL_PATH = "D:/nlp_proj/xlmroberta_ner_results/checkpoint-2225" 

# --- LOAD MODEL ---
print("Loading Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
base_model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT, 
    num_labels=len(NER_TAGS), 
    id2label=id2label, 
    label2id=label2id
)

# Load your LoRA adapters
model = PeftModel.from_pretrained(base_model, LOCAL_MODEL_PATH)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

d:\nlp_proj\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Model...


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on cpu


In [73]:

def get_entities(roman_sentence):
    # 1. Normalize
    clean_roman = normalize_roman_urdu(roman_sentence, FINAL_NORMALIZATION_MAP)
    # 2. Transliterate
    urdu_script = translateToUrdu(clean_roman)
    # 3. Predict
    inputs = tokenizer(urdu_script, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2)
    # 4. Format Output
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    pred_labels = [id2label[p.item()] for p in predictions[0]]
    
    results = []
    for token, label in zip(tokens, pred_labels):
        if token not in ['<s>', '</s>']:
            clean_token = token.replace(' ', ' ')
            # Only show entities, ignore 'O'
            if label != 'O':
                results.append((clean_token, label))
                
    return clean_roman, urdu_script, results

raw_text = "lahor ( ain ain aaii ) adakar wasim abas bhi arzi tor pr krachi shft ho giay"
clean, urdu, entities = get_entities(raw_text)

print(f"Input:    {raw_text}")
print(f"Urdu:     {urdu}")
print("Entities:", entities)

Input:    lahor ( ain ain aaii ) adakar wasim abas bhi arzi tor pr krachi shft ho giay
Urdu:     لاھور ( این این آئی ) اداکار وسیم abas بھی ارضی طور پر کراچی شفٹ حو گئے
Entities: [('▁لا', 'B-Location'), ('ھ', 'I-Location'), ('ور', 'I-Location'), ('▁و', 'B-Person'), ('سیم', 'I-Person'), ('▁a', 'I-Person'), ('bas', 'I-Person'), ('▁کراچی', 'B-Location')]


In [4]:
from normalizer import normalize_roman_urdu, FINAL_NORMALIZATION_MAP
from main import translateToUrdu

In [3]:
pip install translators 

  Using cached translators-6.0.1-py3-none-any.whl.metadata (70 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached niquests-3.15.2-py3-none-any.whl.metadata (16 kB)
  Using cached exejs-0.0.6-py3-none-any.whl.metadata (5.1 kB)
  Using cached pathos-0.3.4-py3-none-any.whl.metadata (11 kB)
  Using cached cloudscraper-1.2.71-py2.py3-none-any.whl.metadata (19 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached cffi-2.0.0-cp313-cp313-win_amd64.whl.metadata (2.6 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached wassima-2.0.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached ppft-1.7.7-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using 


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
